In [113]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [4]:
recipes_data_df = pd.read_csv('out/recipes_data.csv')

In [111]:
recipes_data_df

,title,year
0,terrina de melón con gelée de oporto,1987
1,mousse de trufa negra '87,1987
2,"ensalada de pasta fresca con caviar, tempura d...",1987
3,"raviolis de cigala, patatas y trufa negra",1987
4,tempura de flor de calabacín rellena de mozzar...,1987
5,muslitos de codorniz a la salsa de soja,1987
6,gourmandise de salmón y patata confitada a la ...,1987
7,nido de judías verdes con pinzas de bogavante ...,1987
8,"canapé de patata, salmonete y puré de trufa de...",1987
9,salmonetes Gaudí,1987


In [6]:
recipes_ml_df = pd.read_csv('out/recipes_ml.csv')

In [7]:
recipes_ml_df.head()

,num_ingredients,num_preparations,num_styles,num_techniques,num_techniquesR,num_worlds,i_chocolate troceado,i_pizza,i_jugo de trufa negra,i_fresitas liofilizadas,...,temp_CALIENTE/FRÍA,temp_HELADA/FRÍA,temp_TIBIA/CALIENTE,temp_FRÍA/HELADA,temp_CALIENTE/HELADA,temp_TIBIA,temp_HELADA/FRÍA/AMBIENTE,temp_TIBIA/AMBIENTE,w_DULCE,w_SALADO
0,7,4,1,17,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,15,5,0,22,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
2,21,5,0,29,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,30,10,1,56,0,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
4,25,7,1,37,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [246]:
recipes_data_df[recipes_data_df.year==1997].tail()

,title,year
449,timbal de chocolate con ganache de romero,1997
450,corte de crocant de coco y amaretto,1997
451,rulo de crocant de galleta de chocolate y ment...,1997
452,rulo de crocant de bacon con helado de piñones...,1997
453,virutas bicolores rellenas,1997


In [127]:
X = recipes_ml_df
y = recipes_data_df.year

In [122]:
d={
    1987:'1987-1997',
    1988:'1987-1997',
    1989:'1987-1997',
    1990:'1987-1997',
    1991:'1987-1997',
    1992:'1987-1997',
    1993:'1987-1997',
    1994:'1987-1997',
    1995:'1987-1997',
    1996:'1987-1997',
    1997:'1987-1997',
    1998:'1998-2001',
    1999:'1998-2001',
    2000:'1998-2001',
    2001:'1998-2001',
    2002:'2002-2005',
    2003:'2002-2005',
    2004:'2002-2005',
    2005:'2002-2005',
}

In [124]:
y = [d[x] for x in y]

In [247]:
X = recipes_ml_df.iloc[:454]
y = recipes_data_df.iloc[:454].year

In [248]:
clf = RandomForestClassifier(random_state=0, n_estimators=800, max_depth=20, n_jobs=6)
scores = cross_val_score(clf, X, y, cv=20, scoring=make_scorer(my_diff))
# clf.fit(X, y)

/home/antonio/.virtualenvs/myvirtualenv/lib/python3.5/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 15 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)


In [249]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=6,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [250]:
clf.predict(recipes_ml_df.iloc[1089:])

array([1995, 1994, 1996, 1997, 1996, 1994, 1996, 1996, 1997, 1994, 1996,
       1996, 1996, 1994, 1996, 1994, 1996, 1997, 1994, 1996, 1994, 1996,
       1997, 1996, 1996, 1994, 1997, 1996, 1996, 1994, 1994, 1994, 1994,
       1994, 1995, 1997, 1994, 1994, 1997, 1997, 1997, 1996, 1994, 1997,
       1997, 1996, 1997, 1994, 1996, 1997, 1994, 1997, 1997, 1997, 1994,
       1997, 1997, 1994, 1994, 1994, 1997, 1997, 1994, 1997, 1995, 1997,
       1995, 1996, 1997, 1997, 1995, 1995, 1997, 1997, 1997, 1994, 1997,
       1997, 1994, 1997, 1992, 1992, 1997, 1996, 1997, 1992, 1991, 1992,
       1992, 1992, 1994, 1994, 1994, 1994, 1994, 1994, 1997, 1997, 1997,
       1996, 1996, 1997, 1996, 1997, 1997, 1996, 1997, 1997, 1995, 1996,
       1991, 1996, 1997, 1997, 1994, 1994, 1995, 1994, 1995, 1995, 1995,
       1997, 1994, 1994, 1994])

In [243]:
clf.predict(recipes_ml_df.iloc[1089:])

array([2003, 2003, 2004, 2003, 2004, 2004, 2001, 2000, 2003, 2003, 2003,
       2004, 2003, 2004, 2003, 2004, 2004, 2003, 2003, 2003, 2001, 2003,
       2004, 2004, 2003, 2004, 2003, 2003, 2003, 2003, 2004, 2004, 2003,
       2003, 2003, 2003, 2003, 2003, 2001, 2003, 2004, 2003, 2003, 2003,
       2003, 2004, 2004, 2003, 2004, 2004, 2004, 2004, 2004, 2004, 2003,
       2003, 2004, 2003, 2003, 2003, 2004, 2004, 2004, 2004, 2004, 2003,
       2004, 2004, 2003, 2004, 2004, 2004, 2004, 2004, 2004, 2003, 2003,
       2003, 2004, 2004, 2004, 2003, 2004, 2003, 2003, 2003, 2003, 2003,
       2004, 2003, 2004, 2003, 2001, 2004, 2004, 2003, 2003, 2004, 2004,
       2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2003, 2003,
       2003, 2003, 2003, 2004, 2004, 2004, 2003, 2004, 2003, 2003, 2003,
       2004, 2004, 2004, 2003])

In [244]:
scores

array([ 2.19354839,  1.41666667,  1.5       ,  1.33333333,  1.48333333,
        1.28333333,  1.20689655,  0.98275862,  1.32142857,  1.01818182,
        0.87037037,  1.0754717 ,  0.98076923,  1.        ,  0.90196078,
        1.62      ,  1.28571429,  1.52083333,  1.7173913 ,  1.2826087 ])

In [146]:
clf.predict(recipes_ml_df.iloc[1089:])

array([2003, 2003, 2004, 2003, 2004, 2004, 2001, 2000, 2003, 2003, 2003,
       2004, 2003, 2004, 2003, 2004, 2004, 2003, 2003, 2003, 2001, 2003,
       2004, 2004, 2003, 2004, 2003, 2003, 2003, 2003, 2004, 2004, 2003,
       2003, 2003, 2003, 2003, 2003, 2001, 2003, 2004, 2003, 2003, 2003,
       2003, 2004, 2004, 2003, 2004, 2004, 2004, 2004, 2004, 2004, 2003,
       2003, 2004, 2003, 2003, 2003, 2004, 2004, 2004, 2004, 2004, 2003,
       2004, 2004, 2003, 2004, 2004, 2004, 2004, 2004, 2004, 2003, 2003,
       2003, 2004, 2004, 2004, 2003, 2004, 2003, 2003, 2003, 2003, 2003,
       2004, 2003, 2004, 2003, 2001, 2004, 2004, 2003, 2003, 2004, 2004,
       2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2003, 2003,
       2003, 2003, 2003, 2004, 2004, 2004, 2003, 2004, 2003, 2003, 2003,
       2004, 2004, 2004, 2003])

In [166]:
from sklearn.metrics import make_scorer
from sklearn.metrics import euclidean_distances, accuracy_score
from scipy.spatial.distance import euclidean

In [158]:
accuracy_score([0,0,1],[10,0,1])

0.66666666666666663

In [165]:
euclidean_distances([[0,0,1,2,3],[8,0,2,2,3]])

array([[ 0.        ,  8.06225775],
       [ 8.06225775,  0.        ]])

In [168]:
euclidean([0,0,1,2,3],[8,0,2,2,3])

8.0622577482985491

In [182]:
euclidean([2002, 2003, 2004, 2005], [2003, 2001, 2000, 2005])

4.5825756949558398

In [185]:
euclidean([1987, 1987, 1987, 1987], [2005, 2005, 2005, 2005])

36.0

In [194]:
euclidean([1987, 1987, 1987], [2005, 2005, 2005])/3

10.392304845413264

In [195]:
my_diff([1987, 1987, 1987], [2005, 2005, 2005])/3

18.0

In [190]:
euclidean([1987, 1987], [2005, 2005])

25.45584412271571

In [191]:
my_diff([1987, 1987], [2005, 2005])

36

In [186]:
euclidean([1987], [2005])

18.0

In [207]:
my_diff([1987], [2005])

-1.0

In [187]:
[2,2,2]-[1,0,1]

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [232]:
def my_diff(a,b):
    return (18-sum([abs(x-y) for x,y in zip(a,b)])/len(a))/18

In [240]:
def my_diff(a,b):
    return sum([abs(x-y) for x,y in zip(a,b)])/len(a)

In [233]:
my_diff([1987], [2005])

0.0

In [234]:
my_diff([1995], [2005])

0.4444444444444444

In [235]:
my_diff([2004], [2005])

0.9444444444444444

In [236]:
my_diff([2000, 2004], [2005, 2005])

0.8333333333333334

In [237]:
my_diff([2005, 2005], [2005, 2005])

1.0

In [239]:
my_diff([1987, 1989], [2005, 2005])

17.0

In [180]:
scores

array([-24.69817807, -16.2788206 , -17.54992877, -15.93737745,
       -18.24828759, -15.19868415, -14.28285686, -10.72380529,
       -16.91153453, -13.7113092 , -10.14889157, -13.07669683,
       -11.61895004, -15.45962483, -11.40175425, -18.46618531,
       -16.52271164, -19.        , -20.22374842, -16.40121947])

In [135]:
scores

array([ 0.33333333,  0.40298507,  0.3880597 ,  0.40298507,  0.41791045,
        0.37878788,  0.375     ,  0.484375  ,  0.41935484,  0.47540984,
        0.53333333,  0.50847458,  0.53448276,  0.49122807,  0.45614035,
        0.51785714,  0.50909091,  0.48148148,  0.42307692,  0.51923077])

In [135]:
scores

array([ 0.33333333,  0.40298507,  0.3880597 ,  0.40298507,  0.41791045,
        0.37878788,  0.375     ,  0.484375  ,  0.41935484,  0.47540984,
        0.53333333,  0.50847458,  0.53448276,  0.49122807,  0.45614035,
        0.51785714,  0.50909091,  0.48148148,  0.42307692,  0.51923077])

In [133]:
scores

array([ 0.28985507,  0.43283582,  0.37313433,  0.3880597 ,  0.41791045,
        0.40909091,  0.34375   ,  0.453125  ,  0.43548387,  0.45901639,
        0.51666667,  0.47457627,  0.51724138,  0.47368421,  0.45614035,
        0.51785714,  0.47272727,  0.46296296,  0.36538462,  0.51923077])

In [129]:
scores

array([ 0.31007752,  0.34126984,  0.368     ,  0.46774194,  0.44354839,
        0.39344262,  0.43697479,  0.38135593,  0.37719298,  0.34513274])

In [121]:
scores

array([ 0.30232558,  0.35714286,  0.28      ,  0.2983871 ,  0.30645161,
        0.36065574,  0.42016807,  0.30508475,  0.42105263,  0.27433628])

In [126]:
scores

array([ 0.86178862,  0.94262295,  0.90163934,  0.89344262,  0.95867769,
        0.96694215,  0.92561983,  0.8677686 ,  0.89256198,  0.96666667])

In [118]:
clf.score(X, y)

0.99423393739703458

In [119]:
clf.feature_importances_

array([ 0.01401683,  0.00841437,  0.00500775, ...,  0.        ,
        0.00670966,  0.01149703])